Start with the standard imports we have used for every notebook in this class.

In [29]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

#These are one time load of SCHOOL DISTRICT data steps.
#START SECTION OF DO NOT RUN THESE AGAIN

Each of the datasheets downloaded from ELSI had download metadata on the top of them and total and key information on the bottom of them that were not data rows. This metadata, total, and key information was manually deleted before import. Some of the files had ="0" instead of 0 in the cells. This was found/replaced before import using the sed -i '' 's/="0"/0/g' *.csv command from the terminal.

In [41]:
#CITATION: This is the data from National Center for Education Statistics on Schools
#School districts for all 50 states and Washington, D.C.
#http://nces.ed.gov/ccd/elsi/
#Data Source: U.S. Department of Education National Center for Education Statistics Common Core of Data (CCD) "Local Education Agency (School District) Universe Survey" 2009-10 v.2a  2013-14 v.1a; "Public Elementary/Secondary School Universe Survey" 2009-10 v.2a; "Survey of Local Government Finances School Systems (F-33)" 2009-10 (FY 2010) v.1a.
#KEY:
#† indicates that the data are not applicable.
#– indicates that the data are missing.
#‡ indicates that the data do not meet NCES data quality standards.

districtinformation = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS Information Tab.csv", dtype=np.str)
districtcharacteristicsa = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS CharacteristicsA Tab.csv", dtype=np.str)
districtcharacteristicsb = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS CharacteristicsB Tab.csv", dtype=np.str)
districtenrollments = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS Enrollments Tab.csv", dtype=np.str)
districtenrollmentK3 = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS EnrollK3 Tab.csv", dtype=np.str)
districtenrollment48 = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS Enroll48 Tab.csv", dtype=np.str)
districtenrollment912 = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS Enroll912 Tab.csv", dtype=np.str)
districtteacherstaff = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS TeacherStaff Tab.csv", dtype=np.str)
districtgeneralfinance = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS GeneralFinance Tab.csv", dtype=np.str)
districtrevenue = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS Revenue Tab.csv", dtype=np.str)
districtexpenditures = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS Expenditures Tab.csv", dtype=np.str)

#Data Source: Local Education Agency (School District) Universe Survey Dropout and Completion Data: 2009-10 v.1a.

districtdropoutscompleters = pd.read_csv("tempdata/school districts/2009-2010 DISTRICTS DropoutsCompleters.txt", dtype=np.str, delim_whitespace=True)

Check the lengths of the datasets to see if we have a row for every school district.

In [27]:
print len(districtinformation)
print len(districtcharacteristicsa)
print len(districtcharacteristicsb)
print len(districtenrollments)
print len(districtenrollmentK3)
print len(districtenrollment48)
print len(districtenrollment912)
print len(districtteacherstaff)
print len(districtgeneralfinance)
print len(districtrevenue)
print len(districtexpenditures)
print len(districtdropoutscompleters)

19023
17916
17916
17916
17916
17916
17916
17916
17916
17916
17916
18439


Drop all of the duplicate columns.

In [42]:
districtcharacteristicsb = districtcharacteristicsb.drop(districtcharacteristicsb.columns[[0, 1]], 1)
districtenrollments = districtenrollments.drop(districtenrollments.columns[[0, 1]], 1)
districtenrollmentK3 = districtenrollmentK3.drop(districtenrollmentK3.columns[[0, 1]], 1)
districtenrollment48 = districtenrollment48.drop(districtenrollment48.columns[[0, 1]], 1)
districtenrollment912 = districtenrollment912.drop(districtenrollment912.columns[[0, 1]], 1)
districtteacherstaff = districtteacherstaff.drop(districtteacherstaff.columns[[0, 1]], 1)
districtgeneralfinance = districtgeneralfinance.drop(districtgeneralfinance.columns[[0, 1]], 1)
districtrevenue = districtrevenue.drop(districtrevenue.columns[[0, 1]], 1)
districtexpenditures = districtexpenditures.drop(districtexpenditures.columns[[0, 1]], 1)

Join all of the school district datasets. The datasets districtinformation and districtdropoutscompleters need special treatment, as they have more rows for the school districts than the other datasets. All of the other datasets can be joined by ID without issue.

In [85]:
#Join the datasets that can be joined without issue.
joineddistrict = districtcharacteristicsa.join([districtcharacteristicsb, districtenrollments, districtenrollmentK3, districtenrollment48, districtenrollment912, districtteacherstaff, districtgeneralfinance, districtrevenue, districtexpenditures])

#Clean up an extra hidden character in the Agency Name column
joineddistrict = joineddistrict.rename(columns={'﻿Agency Name': 'Agency Name'})
districtinformation = districtinformation.rename(columns={'﻿Agency Name': 'Agency Name'})

#Merge to the districtinformation dataset
joineddistrict = districtinformation.merge(joineddistrict, 'outer', ['Agency Name', u'State Name [District] Latest available year'], suffixes=('', '_DEL'))

#Need to get rid of Excel syntax ="" from some of the columns
joineddistrict['Agency ID - NCES Assigned [District] Latest available year'] = joineddistrict['Agency ID - NCES Assigned [District] Latest available year'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['County Number [District] 2009-10'] = joineddistrict['County Number [District] 2009-10'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['ANSI/FIPS State Code [District] Latest available year'] = joineddistrict['ANSI/FIPS State Code [District] Latest available year'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['Location ZIP [District] 2013-14'] = joineddistrict['Location ZIP [District] 2013-14'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['Location ZIP4 [District] 2013-14'] = joineddistrict['Location ZIP4 [District] 2013-14'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['Mailing ZIP [District] 2013-14'] = joineddistrict['Mailing ZIP [District] 2013-14'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['Mailing ZIP4 [District] 2013-14'] = joineddistrict['Mailing ZIP4 [District] 2013-14'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['School District Level Code (SCHLEV) [District Finance] 2009-10'] = joineddistrict['School District Level Code (SCHLEV) [District Finance] 2009-10'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['Supervisory Union (ID) Number [District] 2009-10'] = joineddistrict['Supervisory Union (ID) Number [District] 2009-10'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['Metro Micro Area Code [District] 2009-10'] = joineddistrict['Metro Micro Area Code [District] 2009-10'].map(lambda x: str(x).lstrip('="').rstrip('"'))
joineddistrict['Congressional Code [District] 2009-10'] = joineddistrict['Congressional Code [District] 2009-10'].map(lambda x: str(x).lstrip('="').rstrip('"'))
#joineddistrict[''] = joineddistrict[''].map(lambda x: x.lstrip('="').rstrip('"'))

#Rename the LEAID column so it can be merged with the joineddistrict dataset
districtdropoutscompleters = districtdropoutscompleters.rename(columns={'LEAID': 'Agency ID - NCES Assigned [District] Latest available year'})

#Merge to the joineddistrict dataset
joineddistrict = joineddistrict.merge(districtdropoutscompleters, 'outer', 'Agency ID - NCES Assigned [District] Latest available year', suffixes=('', '_DEL'))

#If by chance any rows have NaN, replace with the ELSI standard for missing data '–'
joineddistrict = joineddistrict.fillna('–')
joineddistrict = joineddistrict.replace('nan', '–')

#Where these conditions are true, these rows are not usable
joineddistrict = joineddistrict[joineddistrict['Agency Name']!='–']
joineddistrict = joineddistrict[joineddistrict['Agency Type [District] 2009-10']!='–']

joineddistrict.head()

,Agency Name,State Name [District] Latest available year,State Name [District] 2009-10,State Abbr [District] Latest available year,Agency Name [District] 2009-10,Agency ID - NCES Assigned [District] Latest available year,County Name [District] 2009-10,County Number [District] 2009-10,Race/Ethnicity Category [District] 2009-10,ANSI/FIPS State Code [District] Latest available year,Total Number Operational Schools [Public School] 2009-10,Total Number Operational Charter Schools [Public School] 2009-10,Total Number of Public Schools [Public School] 2009-10,Years District Reported Data [District] Latest available year,Years District Did Not Report Data [District] Latest available year,Location Address [District] 2013-14,Location City [District] 2013-14,Location State Abbr [District] 2013-14,Location ZIP [District] 2013-14,Location ZIP4 [District] 2013-14,Mailing Address [District] 2013-14,Mailing City [District] 2013-14,Mailing State Abbr [District] 2013-14,Mailing ZIP [District] 2013-14,Mailing ZIP4 [District] 2013-14,Phone Number [District] 2013-14,Agency Type [District] 2009-10,School District Level Code (SCHLEV) [District Finance] 2009-10,Urban-centric Locale [District] 2009-10,Boundary Change Indicator Flag [District] 2009-10,CBSA Name [District] 2009-10,CBSA ID [District] 2009-10,CSA Name [District] 2009-10,CSA ID [District] 2009-10,Latitude [District] 2009-10,Longitude [District] 2009-10,State Agency ID [District] 2009-10,Supervisory Union (ID) Number [District] 2009-10,Agency Charter Status [District] 2009-10,Metro Micro Area Code [District] 2009-10,Congressional Code [District] 2009-10,Census ID (CENSUSID) [District Finance] 2009-10,Lowest Grade Offered [District] 2009-10,Highest Grade Offered [District] 2009-10,Total Students (UG PK-12) [District] 2009-10,PK thru 12th Students [District] 2009-10,Ungraded Students [District] 2009-10,Total Students [Public School] 2009-10,Limited English Proficient (LEP) / English Language Learners (ELL) [District] 2009-10,Individualized Education Program Students [District] 2009-10,...,Total - School Admin.- Supp. Serv. Exp. (E09) [District Finance] 2009-10,Total - Student Transp.- Supp. Serv. Exp. (V45) [District Finance] 2009-10,Total - Students- Supp. Serv. Exp. (E17) [District Finance] 2009-10,Salary - Instruction Expenditures (Z33) [District Finance] 2009-10,Salary - Students- Supp. Serv. Exp. (V11) [District Finance] 2009-10,Salary - Instruct. Staff- Supp. Serv. Exp. (V13) [District Finance] 2009-10,Salary - General Admin.- Supp. Serv. Exp. (V15) [District Finance] 2009-10,Salary - School Admin.- Supp. Serv. Exp. (V17) [District Finance] 2009-10,Salary - Ops. & Mainten.- Supp. Serv. Exp. (V21) [District Finance] 2009-10,Salary - Student Transp.- Supp. Serv. Exp. (V23) [District Finance] 2009-10,Salary - Other Supp. Serv.- Supp. Serv. Exp. (V37) [District Finance] 2009-10,Salary - Food Services- Non-Instruction (V29) [District Finance] 2009-10,Employee Benefits - Instruction Expend. (V10) [District Finance] 2009-10,Empl. Benefits - Students- Supp. Serv. Exp. (V12) [District Finance] 2009-10,Empl. Benefits - Instruction- Supp. Serv. Exp. (V14) [District Finance] 2009-10,Empl. Benefits - Gen. Adm.- Supp. Serv. Exp. (V16) [District Finance] 2009-10,Empl. Benefits - Sch. Adm.- Supp. Serv. Exp. (V18) [District Finance] 2009-10,Empl. Benefits - Ops. & Maint.- Supp. Serv. Exp. (V22) [District Finance] 2009-10,Empl. Benefits - Student Trans.- Supp. Serv. Exp. (V24) [District Finance] 2009-10,Empl. Benefits - Other Supp Serv- Supp. Serv. Exp. (V38) [District Finance] 2009-10,Empl. Benefits - Food Services- Non-Instruction (V30) [District Finance] 2009-10,Empl. Benefits - Enterp. Oper.- Non-Instruction (V32) [District Finance] 2009-10,Current Spending - Private Schools (V91) [District Finance] 2009-10,Current Spending - Public Charter Schools (V92) [District Finance] 2009-10,Teacher Salaries - Regular Education Programs (Z35) [District Finance] 2009-10,Teacher Salaries - Special Education Progr

In [86]:
print len(joineddistrict)

18184


In [87]:
joineddistrict.to_csv("tempdata/districts.csv")

#END SECTION OF DO NOT RUN THESE AGAIN

In [3]:
districtsdf = pd.read_csv("tempdata/districts.csv")
districtsdf.head()

C:\Anaconda\lib\site-packages\pandas\io\parsers.py:1170: DtypeWarning: Columns (8,11,35,36,41,463,464,465,466,467,468,469,470) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,Unnamed: 0,Agency Name,State Name [District] Latest available year,State Name [District] 2009-10,State Abbr [District] Latest available year,Agency Name [District] 2009-10,Agency ID - NCES Assigned [District] Latest available year,County Name [District] 2009-10,County Number [District] 2009-10,Race/Ethnicity Category [District] 2009-10,ANSI/FIPS State Code [District] Latest available year,Total Number Operational Schools [Public School] 2009-10,Total Number Operational Charter Schools [Public School] 2009-10,Total Number of Public Schools [Public School] 2009-10,Years District Reported Data [District] Latest available year,Years District Did Not Report Data [District] Latest available year,Location Address [District] 2013-14,Location City [District] 2013-14,Location State Abbr [District] 2013-14,Location ZIP [District] 2013-14,Location ZIP4 [District] 2013-14,Mailing Address [District] 2013-14,Mailing City [District] 2013-14,Mailing State Abbr [District] 2013-14,Mailing ZIP [District] 2013-14,Mailing ZIP4 [District] 2013-14,Phone Number [District] 2013-14,Agency Type [District] 2009-10,School District Level Code (SCHLEV) [District Finance] 2009-10,Urban-centric Locale [District] 2009-10,Boundary Change Indicator Flag [District] 2009-10,CBSA Name [District] 2009-10,CBSA ID [District] 2009-10,CSA Name [District] 2009-10,CSA ID [District] 2009-10,Latitude [District] 2009-10,Longitude [District] 2009-10,State Agency ID [District] 2009-10,Supervisory Union (ID) Number [District] 2009-10,Agency Charter Status [District] 2009-10,Metro Micro Area Code [District] 2009-10,Congressional Code [District] 2009-10,Census ID (CENSUSID) [District Finance] 2009-10,Lowest Grade Offered [District] 2009-10,Highest Grade Offered [District] 2009-10,Total Students (UG PK-12) [District] 2009-10,PK thru 12th Students [District] 2009-10,Ungraded Students [District] 2009-10,Total Students [Public School] 2009-10,Limited English Proficient (LEP) / English Language Learners (ELL) [District] 2009-10,...,Total - School Admin.- Supp. Serv. Exp. (E09) [District Finance] 2009-10,Total - Student Transp.- Supp. Serv. Exp. (V45) [District Finance] 2009-10,Total - Students- Supp. Serv. Exp. (E17) [District Finance] 2009-10,Salary - Instruction Expenditures (Z33) [District Finance] 2009-10,Salary - Students- Supp. Serv. Exp. (V11) [District Finance] 2009-10,Salary - Instruct. Staff- Supp. Serv. Exp. (V13) [District Finance] 2009-10,Salary - General Admin.- Supp. Serv. Exp. (V15) [District Finance] 2009-10,Salary - School Admin.- Supp. Serv. Exp. (V17) [District Finance] 2009-10,Salary - Ops. & Mainten.- Supp. Serv. Exp. (V21) [District Finance] 2009-10,Salary - Student Transp.- Supp. Serv. Exp. (V23) [District Finance] 2009-10,Salary - Other Supp. Serv.- Supp. Serv. Exp. (V37) [District Finance] 2009-10,Salary - Food Services- Non-Instruction (V29) [District Finance] 2009-10,Employee Benefits - Instruction Expend. (V10) [District Finance] 2009-10,Empl. Benefits - Students- Supp. Serv. Exp. (V12) [District Finance] 2009-10,Empl. Benefits - Instruction- Supp. Serv. Exp. (V14) [District Finance] 2009-10,Empl. Benefits - Gen. Adm.- Supp. Serv. Exp. (V16) [District Finance] 2009-10,Empl. Benefits - Sch. Adm.- Supp. Serv. Exp. (V18) [District Finance] 2009-10,Empl. Benefits - Ops. & Maint.- Supp. Serv. Exp. (V22) [District Finance] 2009-10,Empl. Benefits - Student Trans.- Supp. Serv. Exp. (V24) [District Finance] 2009-10,Empl. Benefits - Other Supp Serv- Supp. Serv. Exp. (V38) [District Finance] 2009-10,Empl. Benefits - Food Services- Non-Instruction (V30) [District Finance] 2009-10,Empl. Benefits - Enterp. Oper.- Non-Instruction (V32) [District Finance] 2009-10,Current Spending - Private Schools (V91) [District Finance] 2009-10,Current Spending - Public Charter Schools (V92) [District Finance] 2009-10,Teacher Salaries - Regular Education Programs (Z35) [District Finance] 2009-10,Teacher Salaries - Special Education Programs (Z36) [District Finance] 2009-10,Teacher Salar

#Exported to excel to fix header names and remove duplicates and extra columns... reloaded as districts_cleaned.csv

In [6]:
districtsdf = pd.read_csv("tempdata/districts_cleaned.csv")
print districtsdf.shape
districtsdf.head()

(17537, 342)


,rownum,agency,state,state_abbr,agency_name,agency_id_nces,county,num_schools,num_charter_schools,num_pub_schools,city,zipcode,agency_type,district_level_code,location_type,cbsa_id,latitude,longitude,agency_id_state,charter_status,congressional_code,offered_g_lowest,offered_g_highest,students_ug,total_students,lep,ieps,lunch_free_eligible,lunch_reduced_eligible,lunch_total_free_and_reduced,students_pk_k,students_g1_g8,students_g9_g12,students_pk,students_k,students_g1,students_g2,students_g3,students_g4,students_g5,students_g6,students_g7,students_g8,students_g9,students_g10,students_g11,students_g12,students_ug.1,students_m,students_f,...,fall_membership,totalrev,tlocrev,tsrev,tfedrev,tcurelsc,tcurinst,tcurssvc,tcuroth,tcursalary,tcurbenefits,totalexp,tcapout,tnonelse,arra_rev,arra_exp,arra_capout,totalrev_pp,tlocrev_pp,tsrev_pp,tfedrev_pp,tcurinst_pp,tcurssv_pp,tcuroth_pp,tcursalary_pp,tcurbenefits_pp,totalexp_pp,tcapout_pp,tnonelse_pp,tcurelsc_pp,instexp_pp,tcurelsc_percent,tcurinst_percent,tcuroth_percent,tcurelsc_percent.1,tcurssvc_percent,tfedrev_percent,tlocrev_percent,tsrev_percent,bond_funds,other_funds,survyear,fipst,totd912,ebs912,drp912,totdpl,afgeb,afgr,totohc
0,1,21ST CENTURY CHARTER SCH OF GARY,Indiana,IN,21ST CENTURY CHARTER SCH OF GARY,1800046,MARION COUNTY,1,1,1,GARY,46402,7-Charter school agency,03-Elementary/secondary school system,13-City: Small,26900,39.771949,-86.155184,9545,1-All associated schools are charter schools,1807,Kindergarten,12th Grade,NaN,360,0,28,304,18,322,30,230,100,NaN,30,30,30,30,27,23,27,31,32,31,29,25,15,NaN,186,172,...,360,4000000,128000,2853000,1019000,3290000,1435000,1698000,157000,1488000,370000,4031000,715000,26000,178000,426000,171000,11111,356,7925,2831,3986,4717,436,4133,1028,11197,1986,72,9139,3986,11.2,43.6,4.8,45.2,51.6,25.5,3.2,71.3,0,174000,2009-10,18,-3,100,-3.0,13,43,30.2,-2
1,2,21ST CENTURY CYBER CS,Pennsylvania,PA,21ST CENTURY CYBER CS,4200091,CHESTER COUNTY,1,1,1,EXTON,19341,7-Charter school agency,03-Elementary/secondary school system,21-Suburb: Large,37980,40.005030,-75.678564,124150002,1-All associated schools are charter schools,4206,6th Grade,12th Grade,NaN,594,0,37,111,84,195,0,111,483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,32,60,114,122,138,109,NaN,231,355,...,594,6271000,6109000,162000,0,4816000,3032000,1784000,0,2494000,865000,5187000,329000,0,0,2000,0,10557,10285,273,0,5104,3003,0,4199,1456,8732,554,0,8108,5104,18.0,63.0,0.0,51.8,37.0,0.0,97.4,2.6,0,2132000,2009-10,42,33,483,6.8,102,77,100.0,-2
2,3,21ST CENTURY PREPARATORY SCHOOL AGENCY,Wisconsin,WI,21ST CENTURY PREPARATORY SCHOOL AGENCY,5500045,RACINE COUNTY,1,1,1,RACINE,53404,7-Charter school agency,01-Elementary school system only,13-City: Small,39540,42.729073,-87.797014,8110,1-All associated schools are charter schools,5501,Prekindergarten,8th Grade,NaN,507,35,45,255,35,290,111,396,0,59,52,53,56,51,47,57,45,46,41,NaN,NaN,NaN,NaN,NaN,231,276,...,507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-10,55,-2,-2,-2.0,-2,-2,-2.0,-2
3,5,A CENTER FOR CREATIVE EDUCATION,Arizona,AZ,CENTER FOR CREATIVE EDUCATION INC,400328,YAVAPAI COUNTY,2,2,2,COTTONWOOD,86326,7-Charter school agency,01-Elementary school system only,32-Town: Distant,39140,34.735242,-112.009497,79457,1-All associated schools are charter schools,401,Kindergarten,6th Grade,0,83,0,10,22,12,34,31,52,0,NaN,31,13,5,7,9,8,10,NaN,NaN,NaN,NaN,NaN,NaN,0,49,34,...,83,595000,42000,418000,135000,522000,287000,235000,0,265000,24000,541000,0,19000,0,123000,0,7169,506,5036,1627,3458,2831,0,3193,289,6518,0,229,6289,3458,4.6,55.0,0.0,50.8,45.0,22.7,7.1,70.3,0,0,2009-10,4,-2,-2,-2.0,-2,-2,-2.0,-2
4,6,A CHILD'S VIEW SCHOOL INC.,Arizona,AZ,A CHILD'S VIEW SCHOOL INC.,400431,PIMA COUNTY,1,1,1,TUCSON,85746,7-Charter school agency,01-Elementary school system only,21-Suburb: Large,46060,32.149179,-111.027203,87344,1-All associated schools are charter schools,407,Kin

In [13]:
print districtsdf.totalexp_pp.isnull().sum()
districtsdf.totalexp_pp.count()

1931


15606

In [93]:
districtsdf_gr=districtsdf[districtsdf['afgr']>0]
print districtsdf_gr.shape
districtsdf_gr.head()

(10903, 342)


,rownum,agency,state,state_abbr,agency_name,agency_id_nces,county,num_schools,num_charter_schools,num_pub_schools,city,zipcode,agency_type,district_level_code,location_type,cbsa_id,latitude,longitude,agency_id_state,charter_status,congressional_code,offered_g_lowest,offered_g_highest,students_ug,total_students,lep,ieps,lunch_free_eligible,lunch_reduced_eligible,lunch_total_free_and_reduced,students_pk_k,students_g1_g8,students_g9_g12,students_pk,students_k,students_g1,students_g2,students_g3,students_g4,students_g5,students_g6,students_g7,students_g8,students_g9,students_g10,students_g11,students_g12,students_ug.1,students_m,students_f,...,fall_membership,totalrev,tlocrev,tsrev,tfedrev,tcurelsc,tcurinst,tcurssvc,tcuroth,tcursalary,tcurbenefits,totalexp,tcapout,tnonelse,arra_rev,arra_exp,arra_capout,totalrev_pp,tlocrev_pp,tsrev_pp,tfedrev_pp,tcurinst_pp,tcurssv_pp,tcuroth_pp,tcursalary_pp,tcurbenefits_pp,totalexp_pp,tcapout_pp,tnonelse_pp,tcurelsc_pp,instexp_pp,tcurelsc_percent,tcurinst_percent,tcuroth_percent,tcurelsc_percent.1,tcurssvc_percent,tfedrev_percent,tlocrev_percent,tsrev_percent,bond_funds,other_funds,survyear,fipst,totd912,ebs912,drp912,totdpl,afgeb,afgr,totohc
0,1,21ST CENTURY CHARTER SCH OF GARY,Indiana,IN,21ST CENTURY CHARTER SCH OF GARY,1800046,MARION COUNTY,1,1,1,GARY,46402,7-Charter school agency,03-Elementary/secondary school system,13-City: Small,26900,39.771949,-86.155184,9545,1-All associated schools are charter schools,1807,Kindergarten,12th Grade,NaN,360,0,28,304,18,322,30,230,100,NaN,30,30,30,30,27,23,27,31,32,31,29,25,15,NaN,186,172,...,360,4000000,128000,2853000,1019000,3290000,1435000,1698000,157000,1488000,370000,4031000,715000,26000,178000,426000,171000,11111,356,7925,2831,3986,4717,436,4133,1028,11197,1986,72,9139,3986,11.2,43.6,4.8,45.2,51.6,25.5,3.2,71.3,0,174000,2009-10,18,-3,100,-3.0,13,43,30.2,-2
1,2,21ST CENTURY CYBER CS,Pennsylvania,PA,21ST CENTURY CYBER CS,4200091,CHESTER COUNTY,1,1,1,EXTON,19341,7-Charter school agency,03-Elementary/secondary school system,21-Suburb: Large,37980,40.005030,-75.678564,124150002,1-All associated schools are charter schools,4206,6th Grade,12th Grade,NaN,594,0,37,111,84,195,0,111,483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,32,60,114,122,138,109,NaN,231,355,...,594,6271000,6109000,162000,0,4816000,3032000,1784000,0,2494000,865000,5187000,329000,0,0,2000,0,10557,10285,273,0,5104,3003,0,4199,1456,8732,554,0,8108,5104,18.0,63.0,0.0,51.8,37.0,0.0,97.4,2.6,0,2132000,2009-10,42,33,483,6.8,102,77,100.0,-2
8,10,A+ ACADEMY,Texas,TX,A+ ACADEMY,4800203,DALLAS COUNTY,1,1,1,DALLAS,75217,7-Charter school agency,03-Elementary/secondary school system,11-City: Large,19100,32.767535,-96.660866,57829,1-All associated schools are charter schools,4830,Prekindergarten,12th Grade,NaN,1033,285,63,834,106,940,196,609,228,120,76,68,79,75,63,74,72,86,92,79,45,61,43,NaN,478,555,...,1033,11379000,70000,9466000,1843000,9123000,5051000,3616000,456000,5979000,493000,9156000,0,1000,94000,501000,0,11015,68,9164,1784,4890,3500,441,5788,477,8864,0,1,8832,4890,5.4,55.4,5.0,65.5,39.6,16.2,0.6,83.2,0,2884000,2009-10,48,-3,228,-3.0,39,70,55.7,-2
10,13,A-C CENTRAL CUSD 262,Illinois,IL,A-C CENTRAL CUSD 262,1700105,CASS COUNTY,3,0,3,ASHLAND,62612,1-Local school district,03-Elementary/secondary school system,42-Rural: Distant,NaN,39.892187,-90.016057,46-009-2620-26,3-All associated schools are noncharter,1718,Kindergarten,12th Grade,NaN,432,0,70,107,65,172,29,273,130,1,28,38,34,37,32,27,39,36,30,28,41,31,30,NaN,196,236,...,436,4956000,1907000,2219000,830000,4345000,2108000,2020000,217000,2503000,921000,5755000,961000,0,27000,391000,22000,11367,4374,5089,1904,4835,4633,498,5741,2112,13200,2204,0,9966,4835,21.2,48.5,5.0,57.6,46.5,16.7,38.5,44.8,0,2734000,2009-10,17,-3,130,-3.0,29,41,70.7,0
11,14,A-H-S-T COMM SCHOOL DISTRICT,Iowa,IA,A-H-S-T COMM SCHOOL DISTRICT,1904080,POTTAWATTAMIE COUNTY,2,0,2,AVOCA,51521,1-Local school district,03-Elementary/secondary school system,43-Rural: Remote,36540,41.471017,-95.341001,780

In [ ]:
from scipy.stats.stats import pearsonr
corr_df=[]
corr_fields=[]
for column in districtsdf_gr:
    if(districtsdf_gr[column].dtype == np.float64 or districtsdf_gr[column].dtype == np.int64):
        corr_fields.append(column)
        
print corr_fields[320:]

for i in corr_fields[320:]:   
    for column in districtsdf_gr:
        if(districtsdf_gr[column].dtype == np.float64 or districtsdf_gr[column].dtype == np.int64):
            corr_districtsdf_gr=districtsdf_gr[np.isfinite(districtsdf_gr[column])]
            corr_df.append((column,i,pearsonr(corr_districtsdf_gr[column],corr_districtsdf_gr[i])[0],len(corr_districtsdf_gr)))

In [160]:
corr_data=pd.DataFrame(corr_df)
corr_data.columns=('corrfield1','corrfield2','pearsoncorr','rowcount')
corr_data_filter=(corr_data[corr_data.pearsoncorr >= .80])
corr_data_filter.sort(['pearsoncorr'],ascending=False)

,corrfield1,corrfield2,pearsoncorr,rowcount
2623,totohc,totohc,1.000000,10903
978,ebs912,ebs912,1.000000,10903
320,fipst,fipst,1.000000,10903
2294,afgr,afgr,1.000000,10903
649,totd912,totd912,1.000000,10903
1965,afgeb,afgeb,1.000000,10903
1636,totdpl,totdpl,1.000000,10903
1307,drp912,drp912,1.000000,10903
1,agency_id_nces,fipst,0.999947,10903
9,congressional_code,fipst,0.999937,10903
